# Simphony circuit simulator

[Simphony](https://simphonyphotonics.readthedocs.io/en/latest/) is a circuit simulator based on [scikit-rf](https://scikit-rf.readthedocs.io/en/latest/)

The main advantage of simphony over [SAX](https://flaport.github.io/sax/) is that simphony works in Windows, Linux and MacOs. While SAX only works on MacOs and Linux.

It also supports the SiEPIC PDK library natively.

## Component models

You can use component models from :

- Sparameters from Lumerical FDTD simulations thanks to the gdsfactory Lumerical plugin
- [SiPANN](https://sipann.readthedocs.io/en/latest/?badge=latest) open source package

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gdsfactory as gf

import gdsfactory.simulation.simphony as gs
import gdsfactory.simulation.simphony.components as gc

In [ ]:
c = gf.components.mzi()
n = c.get_netlist()

### Straight

Lets start with the Sparameter model of a straight waveguide.

The models are for lossless elements.

In [ ]:
m = gc.straight()
wavelengths = np.linspace(1500, 1600, 128) * 1e-9
gs.plot_model(m, logscale=False, wavelengths=wavelengths)

In [ ]:
m = gc.straight()
wavelengths = np.linspace(1500, 1600, 128) * 1e-9
gs.plot_model(m, phase=True, wavelengths=wavelengths)

### Bend

In [ ]:
m = gc.bend_circular(radius=2)  # this bend should have some loss
gs.plot_model(m, logscale=False, wavelengths=wavelengths)

In [ ]:
m = gc.mmi1x2()  # this model comes from Lumerical FDTD 3D sims
gs.plot_model(m, pin_in="o1")

In [ ]:
m = gc.mmi1x2()
gs.plot_model(m, pin_in="o1", phase=True)
plt.xlim([1540, 1570])

In [ ]:
m.pins

As you can see the MMI has -20dB reflection and -3dB transmission

In [ ]:
gs.plot_model(m, pins=("o2", "o3"))

In [ ]:
m.pins

In [ ]:
m = gc.mmi2x2()  # this model comes from Lumerical FDTD 3D sims
gs.plot_model(m)

In [ ]:
gs.plot_model(m, pins=("o3", "o4"))

In [ ]:
m = gc.coupler_ring()
gs.plot_model(m, logscale=False, wavelengths=wavelengths)

In [ ]:
gc.coupler_ring?

In [ ]:
m = gc.coupler_ring(gap=0.3)
gs.plot_model(m, logscale=False, wavelengths=wavelengths)

In [ ]:
m = gc.coupler(gap=0.3)
gs.plot_model(m, logscale=False, wavelengths=wavelengths)

In [ ]:
m = gc.gc1550te()
gs.plot_model(m, logscale=True, pin_in="port 1")

In [ ]:
m = gc.gc1550te()
gs.plot_model(m, logscale=True, pin_in="port 1")
plt.ylim([-10, -3])

In [ ]:
m = gc.gc1550te()
gs.plot_model(m, logscale=False, pin_in="port 1")

## Circuit simulations

With Simphony you can also combine components into circuits

### MZI interferometer

In [ ]:
import matplotlib.pyplot as plt
import gdsfactory.simulation.simphony as gs
import gdsfactory.simulation.simphony.components as gc
import gdsfactory as gf

In [ ]:
c = gf.components.mzi(delta_length=10)
c

In [ ]:
c.plot_netlist()

In [ ]:
circuit = gs.components.mzi(delta_length=10, splitter=gs.components.mmi1x2)

gs.plot_circuit(
    circuit,
    start=1500e-9,
    stop=1600e-9,
    logscale=True,
)

In [ ]:
circuit = gs.components.mzi(delta_length=100, splitter=gs.components.mmi1x2)

gs.plot_circuit(
    circuit,
    start=1500e-9,
    stop=1600e-9,
    logscale=True,
)

Lets add grating couplers to the mzi circuit.

In [ ]:
mzi_layout = gf.components.mzi(delta_length=100)
mzi_with_gc_layout = gf.routing.add_fiber_single(
    component=mzi_layout, with_loopback=False
)
mzi_with_gc_layout

In [ ]:
c = gc.gc1550te()
gs.plot_model(c, pin_in="port 1")

In [ ]:
circuit_gc = gs.add_gc(circuit=circuit, gc=gs.components.gc1550te)

gs.plot_circuit(
    circuit_gc,
    start=1520e-9,
    stop=1580e-9,
    logscale=True,
)

### MZI intereferometer from layout

In [ ]:
from simphony.library import siepic
from simphony.netlist import Subcircuit
import gdsfactory as gf

import gdsfactory.simulation.simphony as gs

c = gf.components.mzi(delta_length=10)
cm = gs.component_to_circuit(c)

In [ ]:
cm.pins

In [ ]:
p2 = cm.pins.pop()
p2.name = "o2"

In [ ]:
cm.pins

In [ ]:
gs.plot_circuit(cm)

### Ring resonator


In [ ]:
c = gc.ring_double(radius=5)
gs.plot_circuit(c, pins_out=["o2", "o3", "o4"])

In [ ]:
c = gs.components.ring_double(radius=10)
gs.plot_circuit(c, pins_out=["o2", "o3", "o4"])